### 1-D Convolutional Neural Networks with Keras

In this notebook we show how to run a simple deep learning model to classify whale calls. We use the [keras](https://keras.io/) library. We will use a convolutional neural networks, since they are robust to temporal and spatial shifts. Although it is common to use 2D convolutional neural networks when working with spectrogram, we will use a 1D on the time series, as this can be applied to other time series classification problems. But instead of raw data, we will use the power spectrum profile of each signal and we hope that they will be sufficiently different for signals which contain right whale calls.

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# importing multiple visualization libraries
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import mlab
import pylab as pl
import seaborn as sns

In [ ]:
# importing libraries to manipulate the data files
import os
from glob import glob

In [ ]:
# import a library to read the .aiff sound format
import aifc

**Preparing the Dataset**

In [ ]:
# reading the filenames
filenames = sorted(glob(os.path.join('data','train','*.aiff')))

In [ ]:
from scipy import signal

We want to calculate the power spectrum for each signal (which will be a one dimensional object). We can calculate tit using the [welch filter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html#scipy.signal.welch) from the `scipy.signal` library.

In [ ]:
# read signals and apply the welch filter
feature_dict = {}
fs = 2000
for filename in filenames[:10000]:
    aiff = aifc.open(filename,'r')
    whale_strSig = aiff.readframes(aiff.getnframes())
    whale_array = np.fromstring(whale_strSig, np.short).byteswap()
    # apply welch filter
    feature = 10*np.log10(signal.welch(whale_array, fs=fs, window='hanning', nperseg=256, noverlap=128+64)[1])
    feature_dict[filename] = feature

In [ ]:
# plot an example of power spectrum
plt.plot(feature_dict['data/train/train00006.aiff'])
plt.title('Power Spectrum for a Right Whale Up-call')
plt.ylabel('Power Spectrum')
plt.xlabel('Time in Seconds')

In [ ]:
#y = np.array(labels['label'][X.columns])
y = np.load('y.npy')
y = y.astype('float32')

In [ ]:
X = pd.DataFrame(feature_dict).T

**Importing Libraries for Deep Learning on Time Domain Objects**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard

**Training - Validation Dataset Split**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=2018, shuffle=True)

X_train = np.expand_dims(X_train, axis=2)
X_val = np.expand_dims(X_val, axis=2)

**Building the Neural Network**

In [ ]:
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(129, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

**Jargon Translation:**

|Term| Explanation|
|---|---|
|Convolutional Layers | layers which are robust to time shifts|
|Max Pooling/ Average Pooling |dimension reduction, robustness|
|Batch normalization |equalizes the distribution of the batches |
|Epochs |1 run of a batch |
|Adam Optimizer | an adaptive optimization scheme|
|Cross Entropy | cost function|
|ReLU | nonlinear activation function| 
|Batch Size| size of subset to process to update the estimates|
|Learning Rate | time step of the optimization algorithm|

**Visualizing the Computation Graph**

In [ ]:
# from keras.utils import plot_model
# plot_model(model,to_file='model_plot.png')

<img src="img/model_plot.png" width=200/>

**Model Logging Object**

In [ ]:
model_name = 'deep_1'
top_weights_path = 'model_' + str(model_name) + '.h5'

callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True), 
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]

**Model Training**

In [ ]:
%%time
# Fitting the Model(this will take loooooooong time, if run on CPU)
history = model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data = [X_val, y_val], callbacks = callbacks_list)

**Evaluation**

In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_val, y_val, batch_size=16)
print('Validation accuracy:', acc)

In [ ]:
# prediction and prediction scores
y_pred = np.array((model.predict(X_val)>0.5).astype('int32'))
y_score = model.predict_proba(X_val)

In [ ]:
# precision
print('Validation precision:', sum(y_pred[y_val == 1]>0)/sum(y_pred == 1))

In [ ]:
# recall
print('Validation recall:', sum(y_pred[y_val == 1]>0)/sum(y_val == 1))

In [ ]:
# confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_val,y_pred)
# Transform to df for easier plotting
cm_df = pd.DataFrame(cm,
                     index = ['No_Upcall','Upcall'], 
                     columns = ['No_Upcall','Upcall'])

plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, fmt='g')
plt.title('\nAccuracy:{0:.3f}'.format(accuracy_score(y_val, y_pred)))
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
# Model Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Prediction Accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Prediction Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
# Accuracy
from sklearn.metrics import average_precision_score, roc_auc_score, roc_curve 
roc_auc = roc_auc_score(y_val.astype('int'), y_score)
fpr, tpr, _ = roc_curve(y_val, y_score)

In [ ]:
# Plotting Accuracy
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate\n predicted upcall when no upcall')
plt.ylabel('True Positive Rate\n predicted upcall when upcall')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

In [ ]:
# Precision Recall
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_val.astype('int'), y_score)
print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
# Plotting Prediction and Recall
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(y_val.astype('int'), y_score)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

**More:** 

* Data Augmentation: [Image Data Generators](https://keras.io/preprocessing/image/) 
* Transfer Learning: [blog](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a), [example](https://www.kaggle.com/valcoder/whalesounds-dl-2d/)
* 2D Convolutional Networks

**References:**

[Deep Learning Glossary](http://www.wildml.com/deep-learning-glossary/)

[Keras and NN Tutorial](https://indico.cern.ch/event/506145/contributions/2132944/attachments/1258124/1858154/NNinKeras_MPaganini.pdf)

[Keras Cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Keras_Cheat_Sheet_Python.pdf)

Free GPU usage: [Google Colaboratory notebooks](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true) & [Kaggle Kernels](https://www.kaggle.com/kernels).

Monitoring Performance: [TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard), [TensorWatch](https://github.com/microsoft/tensorwatch/blob/master/README.md)